In [1]:
from sympy import *
import numpy as np

from numpy import concatenate
from scipy.linalg import block_diag


# Map python functions to MATAB syntax where possible

In [2]:
def ones(*args):
    return np.ones(args, dtype=int)
def zeros(*args):
    return np.zeros(args, dtype=int)
def eye(n, m=None, k=0):
    if isinstance(n, tuple):
        m = n[1]
        n = n[0]
    return np.eye(n,m,k, dtype=int)

catr = lambda x: concatenate(x, 0)
catc = lambda x: concatenate(x, 1)
size = lambda x: np.shape(x)

# Generate the needed symbolic variables

In [3]:
# shorten the number of variables for readability
T = 3
K = 2

C_k = list(symbols('C_1:%d'%(K+1)))
V_sk_max = list(symbols('V_s1:%d_max'%(K+1)))
R_sk_max = list(symbols('R_s1:%d_max'%(K+1)))
Delta = symbols('Delta')


sum_I_Mn = list(symbols('sum_I_Mn_1:%d'%(T+1)))

I_b = list(symbols('I_b_1:%d'%(T+1)))

I_out = []
I_in = []
V_s = []
L = []

for k in range(K):
    I_out += list(symbols('I_ou_%d_1:%d'%(k+1,T+1)))
    I_in += list(symbols('I_in_%d_1:%d'%(k+1,T+1)))
    V_s += list(symbols('V_s_%d_0:%d'%(k+1,T+1)))
    L += list(symbols('L_%d_1:%d'%(k+1,T+1)))

x = I_b + I_out + I_in + V_s + L

In [4]:
sum_I_Mn = Matrix(sum_I_Mn)
I_b = Matrix(I_b)
I_in = Matrix(I_in)
I_out = Matrix(I_out)
V_s = Matrix(V_s)
L = Matrix(L)
x = Matrix(x)
x.T

Matrix([[I_b_1, I_b_2, I_b_3, I_ou_1_1, I_ou_1_2, I_ou_1_3, I_ou_2_1, I_ou_2_2, I_ou_2_3, I_in_1_1, I_in_1_2, I_in_1_3, I_in_2_1, I_in_2_2, I_in_2_3, V_s_1_0, V_s_1_1, V_s_1_2, V_s_1_3, V_s_2_0, V_s_2_1, V_s_2_2, V_s_2_3, L_1_1, L_1_2, L_1_3, L_2_1, L_2_2, L_2_3]])

# constraint 1

In [5]:
"""
% constraint : I_B + sum_k(I_sk_out - I_sk_in) - sum_n(I_Mn) = 0

% eq_lh = I_B + sum_k(I_sk_out - I_sk_in);
% eq_rh = sum_k(I_Mn, 2);

% in matrix form this should be:
% [I,   I,    I,    I,    I,    -I,   -I,   -I,   -I] * 
% [I_b, I_1o, I_2o, I_3o, I_4o, I_1i, I_2i, I_3i, I_4i]^T
"""

I = eye(T, T);
A1 = eye(T, T);
A4 = zeros(T, (T+1)*K);
A5 = zeros(T, T*K);

for k in range(K):
    A2 = catc([A2,  eye(T, T)]) if k > 0 else eye(T, T);
    A3 = catc([A3, -eye(T, T)]) if k > 0 else -eye(T, T);
    
#        I_b, I_sk_out, I_sk_in,  V_sk, L_k
eq1_A = catc([A1,  A2,       A3,      A4,   A5]);
eq1_b = sum_I_Mn;

eq1_A * x

Matrix([
[I_b_1 - I_in_1_1 - I_in_2_1 + I_ou_1_1 + I_ou_2_1],
[I_b_2 - I_in_1_2 - I_in_2_2 + I_ou_1_2 + I_ou_2_2],
[I_b_3 - I_in_1_3 - I_in_2_3 + I_ou_1_3 + I_ou_2_3]])

In [6]:
eq1_b

Matrix([
[sum_I_Mn_1],
[sum_I_Mn_2],
[sum_I_Mn_3]])

This looks good and seems to **match the paper**. The equation given in the paper:

\begin{equation*}
I_B + \sum_{k \in K} \left( I_{S_k}^{out} - I_{S_k}^{in} \right) - \sum_{n \in N} I_{M_n} = \boldsymbol{0}
\end{equation*}

# constraint 2 implementation from paper in P1

In [14]:
"""
% constraint : (A-I) * V_sk - D_k_out * I_sk_out - D_k_in * I_sk_in = 0

% eq_lh = (A-I) * V_sk - D_k_out * I_sk_out - D_k_in * I_sk_in;
% eq_rh = 0;

% should in matrix form be 
% [0, A-I, -D_k_out, -D_k_in, 0] *
% [I_b, V_sk, I_sk_out, I_sk_in, L_k]^T
% where the non zero terms must actually be concat k times
"""

# part 1: (A-I) * V_sk
A11 = zeros(1, T);
A11[0,0] = 1;
A12 = zeros(1, 1);
A21 = eye(T);
A22 = zeros(T,1);

A1 = catc([A11, A12])
A2 = catc([A21, A22])
A_k = catr([A1, A2]);


# build block diagonalmatrix with k blocks on diagonal

A = block_diag(*[A_k for _ in range(K)])
Ia = eye(size(A));

# part 2: D_k_out * I_sk_out - D_k_in * I_sk_in
tmp = catr([ zeros(1,T), eye(T)]);

# stack K times

c1, c2 = symbols('c1 c2')

D_out = block_diag(*[c1 * tmp for _ in range(K)])
D_in = block_diag(*[c2 * tmp for _ in range(K)])

Z = zeros(D_out.shape[0], T)
Z_k = zeros(D_out.shape[0], K*T)

print((size(D_out), size(D_in), size((A-Ia))))
#         I_b,  I_sk_out, I_sk_in,  V_sk,    L_k
eq2_A = catc([ Z,    -D_out,   -D_in,    (A-Ia), Z_k]);

eq2_b = Matrix(zeros(T+1, 1));

-1 * (eq2_A * x)

((8, 6), (8, 6), (8, 8))


Matrix([
[                                            0],
[I_in_1_1*c2 + I_ou_1_1*c1 - V_s_1_0 + V_s_1_1],
[I_in_1_2*c2 + I_ou_1_2*c1 - V_s_1_1 + V_s_1_2],
[I_in_1_3*c2 + I_ou_1_3*c1 - V_s_1_2 + V_s_1_3],
[                                            0],
[I_in_2_1*c2 + I_ou_2_1*c1 - V_s_2_0 + V_s_2_1],
[I_in_2_2*c2 + I_ou_2_2*c1 - V_s_2_1 + V_s_2_2],
[I_in_2_3*c2 + I_ou_2_3*c1 - V_s_2_2 + V_s_2_3]])


whith $c1 = R_{sk_{max}} + \Delta / C_k$ and $c2 = R_{sk_{max}} - \Delta / C_k$

This is the Matrix equation from P1, and **maches** the equation in the paper:

\begin{equation*}
v_{s_k}\left( t \right) = -\sum_{\tau=1}^t \left( \frac{\Delta}{C_k} \cdot i_{s_k}  \left( \tau \right) + R_{s_k} \cdot \left| i_{s_k} \left( \tau \right) \right| \right)
\end{equation*}

which is equal to:

\begin{equation*}
v_{s_k}\left( t \right) =  v_{s_k}\left( t-1 \right) - \left( \frac{\Delta}{C_k} \cdot i_{s_k}  \left( t \right) + R_{s_k} \cdot \left| i_{s_k} \left( t \right) \right| \right)
\end{equation*}

and by the steps given in the paper we get to:

\begin{equation*}
v_{s_k}\left( t \right) =  v_{s_k}\left( t-1 \right) - \left( c_1 i_{out_{k}} \left( t \right)  + c_2 i_{in_{k}} \left( t \right) \right)
\end{equation*}

the last equation however only seems to hold while $I_out < I_in$, which is omitted in the paper and might cause complications.

In [ ]:
eq2_b

In [ ]:
A * V_s

In [ ]:
D_out * I_out

In [ ]:
D_in * I_in

# constraint 3

In [ ]:
# constraint : E * V_sk = 0

# eq_lh = E * V_sk
# eq_rh = 0;

E_sub = zeros(1, T+1);
E_sub[0,0] = 1;
E_sub[0,-1] = -1;

# stack K times
for k in range(K):
    E = catc([E, E_sub]) if k > 0 else E_sub;



Z1 = zeros(1, T);
Z2 = zeros(1, K*T);

#         I_b,  I_sk_out,   I_sk_in, V_sk, L_k
eq3_A = catc([ Z1,   Z2,         Z2,       E,   Z2 ]);
eq3_b = Matrix([[0]]);

eq3_A * x

In [ ]:
eq3_b

This **matches** the given equation:

\begin{equation*}
v_{s_k}\left( 0 \right) = v_{s_k}\left( T \right)
\end{equation*}


# combine all constraints to one 

In [ ]:
Aeq = catr([eq1_A, eq2_A, eq3_A]);

# test for debugging --> this must not fail
Aeq * x


In [ ]:
beq = Matrix(catr([eq1_b,eq2_b,eq3_b]))
beq

# inequality constraint 1

In [ ]:
"""
%% inequality constraint 1

% constraint : -L_k <= I_sk_out - I_sk_in <= L_k
% can be rewritten into: 
%   (1): -L_k <= I_sk_out - I_sk_in
%   (2): I_sk_out - I_sk_in <= L_k      | switch sides
% can be rewritten into: 
%   (1): -L_k <= I_sk_out - I_sk_in
%   (2):  L_k >= I_sk_out - I_sk_in     | * -1
% can be rewritten into: 
%   (1): -L_k <=  I_sk_out - I_sk_in    | -I_sk_out | +I_sk_in
%   (2): -L_k <= -I_sk_out + I_sk_in    | +I_sk_out | -I_sk_in
%  which can be rewritten to:
%   (1): -I_sk_out + I_sk_in - L_k <= 0
%   (2):  I_sk_out - I_sk_in - L_k <= 0
% which in matrix algebra is:
"""

Z          = zeros(T*K, T);
Z_k        = zeros(T*K, K*(T+1));

I = eye(T)
E_I_sk_out = eye(T*K);
E_I_sk_in = eye(T*K);
E_L_k = eye(T*K);

#      I_b,  I_sk_out,    I_sk_in,   V_sk,  L_k
A_11 = catc([Z,   -E_I_sk_out,  E_I_sk_in, Z_k,  -E_L_k]);
b_11 = zeros(T*K, 1);

#      I_b,  I_sk_out,    I_sk_in,   V_sk,  L_k
A_12 = catc([Z,    E_I_sk_out, -E_I_sk_in, Z_k,  -E_L_k]);
b_12 = zeros(T*K, 1);

#  combine all inequality constraints
A = catr([A_11, A_12]);

b = catr([b_11, b_12]);
A = Matrix(A)
b = Matrix(b)

# A * x <= b

A * x

In [ ]:
b

This **matches** the equation given in the paper:

\begin{equation*}
-L_k \leq I_{S_k}^{out} - I_{S_k}^{in} \leq L_k
\end{equation*}
